<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/Paper(1_s2_0_S0957417422020255_main)_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install Required Libraries
!pip install numpy pandas tensorflow keras gensim nltk scikit-learn openpyxl


In [ ]:

# Step 2: Import Libraries
import numpy as np
import pandas as pd
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D, Dense, Dropout, Input, Bidirectional, Attention, GlobalAveragePooling1D, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:


# Step 3: Load and Preprocess Data
# Load the Excel file
from google.colab import files
uploaded = files.upload()  # Manually upload your .xlsx file

# Change the file name as per the uploaded file
df = pd.read_excel(next(iter(uploaded.keys())))  # Replace with 'filename.xlsx' if needed


Saving Bangla2.xlsx to Bangla2.xlsx


In [ ]:
# df = df.sample(frac=0.4, random_state=42)
print(df.head())

                                              tweets  labels
0  জনগণের ভোট চুরি করে ক্ষমতায় আসা ওই নিয়ত আমাদ...       0
1    সবথেকে বড় যুদ্ধ হলো নিজের মনের সাথে যুদ্ধ আবেগ       0
2  তোমাকে মনে পড়বে যখনিই জ্যোস্না হাসে তোমাকে মন...       0
3  রোজ আকাশে একটা তারা জলমল করতো হাজার মেঘলা এমনক...       0
4  আল্লাহ মহাপরাক্রমশালী এবং সবার ওপর পরিপূর্ণরূপ...       0


In [ ]:

# Define the text and label columns
text_column_name = 'tweets'  # replace with actual column name for text
label_column_name = 'labels'  # replace with actual column name for labels


In [ ]:

# Preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(str(text).lower())
    # words = [word for word in tokens if word.isalpha()]
    words = [word for word in words if word not in stopwords.words('bengali')]
    return " ".join(words)

df[text_column_name] = df[text_column_name].apply(preprocess_text)

# Step 4: Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(df[text_column_name], df[label_column_name], test_size=0.2, random_state=42)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

fasttext_model = gensim.models.KeyedVectors.load('/content/drive/MyDrive/embeddings/fasttext_bn_model.kv')


In [ ]:
from gensim.models import KeyedVectors
from google.colab import drive
from gensim.scripts.glove2word2vec import glove2word2vec

# # Load the GloVe embeddings
# glove_model_path = '/content/drive/MyDrive/bn_glove.39M.300d.txt'
# word2vec_output_file = '/content/GloVe-Bengali/word2vec_bengali.txt'

# # Convert GloVe format to Word2Vec format
# glove2word2vec(glove_model_path, word2vec_output_file)



In [ ]:
# glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


In [ ]:

# # Save the model to Google Drive in binary format
# glove_model_save_path = '/content/drive/MyDrive/embeddings/glove_bn_model.kv'
# os.makedirs(os.path.dirname(glove_model_save_path), exist_ok=True)

# glove_model.save(glove_model_save_path)

In [ ]:
glove_model_path = '/content/drive/MyDrive/embeddings/glove_bn_model.kv'
glove_model = KeyedVectors.load(glove_model_path)

In [ ]:
# !git clone https://github.com/sagorbrur/bnlp.git
# #

Cloning into 'bnlp'...
remote: Enumerating objects: 2130, done.
remote: Counting objects: 100% (653/653), done.
remote: Compressing objects: 100% (322/322), done.
remote: Total 2130 (delta 387), reused 535 (delta 326), pack-reused 1477 (from 1)
Receiving objects: 100% (2130/2130), 22.76 MiB | 26.10 MiB/s, done.
Resolving deltas: 100% (1265/1265), done.


In [ ]:
!pip install bnlp_toolkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=abf60055a10b156168e1b94df5d796a5b1a810fc05b73854a4f69a697374be02
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: tqdm
    Found exist

In [ ]:
from bnlp import BengaliWord2Vec
from gensim.models import KeyedVectors
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, Concatenate
from tensorflow.keras.models import Model

# Initialize BengaliWord2Vec with pretrained model
bwv = BengaliWord2Vec()

# Define function to retrieve word vector embeddings for BengaliWord2Vec
def get_bwv_embedding_matrix(word_index):
    """
    Creates an embedding matrix for the BengaliWord2Vec model.

    Args:
        word_index (dict): A dictionary mapping words to their indices.

    Returns:
        np.ndarray: The embedding matrix.
    """
    embedding_dim = 100  # Assuming a dimension of 300 for BengaliWord2Vec
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        try:
            vector = bwv.get_word_vector(word)
            if vector is not None and len(vector) > 0:
                embedding_matrix[i] = vector
        except KeyError:
            print(f"Warning: Word '{word}' not found in the BengaliWord2Vec vocabulary. Skipping.")
    return embedding_matrix

# Define function to retrieve word vector embeddings for other models
def get_embedding_matrix(word_index, embedding_model, embedding_dim=300):
    """
    Creates an embedding matrix for a given embedding model.

    Args:
        word_index (dict): A dictionary mapping words to their indices.
        embedding_model (object): The word embedding model.
        embedding_dim (int): The desired embedding dimension.

    Returns:
        np.ndarray: The embedding matrix.
    """
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
    return embedding_matrix

# Tokenize the input data
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

padded_train = pad_sequences(train_sequences, maxlen=100)
padded_test = pad_sequences(test_sequences, maxlen=100)
word_index = tokenizer.word_index

# Step 4: Create embedding matrices for each model
embedding_matrix_bwv = get_bwv_embedding_matrix(word_index)
embedding_matrix_ft = get_embedding_matrix(word_index, fasttext_model, embedding_dim=300)
embedding_matrix_glove = get_embedding_matrix(word_index, glove_model, embedding_dim=300)

# Step 5: Build the Hybrid Model
# Input layer
input_layer = Input(shape=(100,))

# Embedding layers for each embedding type
embedding_layer_bwv = Embedding(len(word_index) + 1, 100, weights=[embedding_matrix_bwv], trainable=False)(input_layer)
embedding_layer_ft = Embedding(len(word_index) + 1, 300, weights=[embedding_matrix_ft], trainable=False)(input_layer)
embedding_layer_glove = Embedding(len(word_index) + 1, 300, weights=[embedding_matrix_glove], trainable=False)(input_layer)

# Concatenate embeddings
concatenated_embeddings = Concatenate()([embedding_layer_bwv, embedding_layer_ft, embedding_layer_glove])


# Continue building the model as needed...


In [ ]:
print(concatenated_embeddings)

<KerasTensor shape=(None, 100, 700), dtype=float32, sparse=False, name=keras_tensor_20>


In [ ]:
# Check dimension for each model to ensure compatibility
fasttext_dim = len(fasttext_model[next(iter(fasttext_model.key_to_index))])  # For gensim FastText model
glove_dim = len(glove_model[next(iter(glove_model.key_to_index))])          # For gensim GloVe model

# Verify dimensions
print(f"FastText embedding dimension: {fasttext_dim}")
print(f"GloVe embedding dimension: {glove_dim}")

FastText embedding dimension: 300
GloVe embedding dimension: 300


In [ ]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Attention
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Assuming `input_layer` and `concatenated_embeddings` are already defined
# Layer definitions
bilstm_out = Bidirectional(LSTM(100, return_sequences=True))(concatenated_embeddings)
lstm_out = LSTM(100, return_sequences=True)(bilstm_out)
cnn_out = Conv1D(filters=50, kernel_size=3, activation='relu')(lstm_out)

attention_out = Attention()([cnn_out, cnn_out])  # Adjusted for attention layer

# Global max pooling and dense layers
max_pool_out = GlobalMaxPooling1D()(attention_out)  # Applying GlobalMaxPooling1D after attention
dropout_layer_1 = Dropout(0.5)(max_pool_out)
dense_layer_1 = Dense(250, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dropout_layer_2)  # Use sigmoid for binary classification

# Define and compile model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use binary crossentropy for binary classification
model.summary()

# Step 8: Train the Model
model.fit(padded_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Step 9: Evaluate the Model
predictions = model.predict(padded_test)
predicted_classes = (predictions > 0.5).astype("int32")  # Get the predicted class indices based on a threshold
print("Accuracy:", accuracy_score(y_test, predicted_classes))
print("AUC:", roc_auc_score(y_test, predictions))  # For binary classification, use the predictions directly
print("Classification Report:\n", classification_report(y_test, predicted_classes))


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_5 (Embedding)   │ (None, 100, 100)       │         27,500 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_6 (Embedding)   │ (None, 100, 300)       │         82,500 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, 100, 300)       │         82,500 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 100, 700)       │              0 │ embedding_5[0][0],     │
│ (Concatenate)             │                        │                │ embedding_6[0][0],     │
│                           │                        │                │ embedding_7[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_1           │ (None, 100, 200)       │        640,800 │ concatenate_1[0][0]    │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ (None, 100, 100)       │        120,400 │ bidirectional_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 98, 50)         │         15,050 │ lstm_3[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_1 (Attention)   │ (None, 98, 50)         │              0 │ conv1d_1[0][0],        │
│                           │                        │                │ conv1d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_1    │ (None, 50)             │              0 │ attention_1[0][0]      │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_2 (Dropout)       │ (None, 50)             │              0 │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 250)            │         12,750 │ dropout_2[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_3 (Dropout)       │ (None, 250)            │              0 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 1)              │            251 │ dropout_3[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 981,751 (3.75 MB)

 Trainable params: 789,251 (3.01 MB)

 Non-trainable params: 192,500 (751.95 KB)

Epoch 1/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 31ms/step - accuracy: 0.7436 - loss: 0.6205 - val_accuracy: 0.7611 - val_loss: 0.5421
Epoch 2/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - accuracy: 0.7426 - loss: 0.5568 - val_accuracy: 0.7611 - val_loss: 0.5391
Epoch 3/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.7493 - loss: 0.5481 - val_accuracy: 0.7452 - val_loss: 0.5419
Epoch 4/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7551 - loss: 0.5324 - val_accuracy: 0.7548 - val_loss: 0.5372
Epoch 5/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - accuracy: 0.7574 - loss: 0.5261 - val_accuracy: 0.7516 - val_loss: 0.5386
Epoch 6/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.7578 - loss: 0.5167 - val_accuracy: 0.7643 - val_loss: 0.5385
Epoch 7/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 67ms/step - accuracy: 0.7642 - loss: 0.5106 - val_accuracy: 0.7484 - val_loss: 0.5566
Epoch 8/10
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 21ms/step - accuracy: 0.7669 - loss: 0.5003 - val_accuracy: 0.7484 - v